<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/semantic_vector6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=4baeb7dc0c755d2a583aabc961ce96b78f853a7839b18e05b10d002fee5d0f40
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import numpy as np
import re
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Read log templates file into a DataFrame
df = pd.read_csv('/content/BGL_templates.csv')
df.head(3)

,EventId,EventTemplate
0,E1,<*>:<*> <*>:<*> <*>:<*> <*>:<*>
1,E2,<*> <*> <*> BGLERR_IDO_PKT_TIMEOUT connection ...
2,E3,<*> correctable errors exceeds threshold (iar ...


In [40]:
stop_words =['a','about','above','after','again','against','ain','all','am','an','and','any','are','aren',"aren't",'as','at','be','because','been','before','being','below','between','both',
             'but','by','can','couldn',"couldn't",'d','did','didn',"didn't",'do','does','doesn',"doesn't",'doing','don',"don't",'during','each','few','for','from','further','had',
             'hadn',"hadn't",'has','hasn',"hasn't",'have','haven',"haven't",'having','he','her','here','hers','herself','him','himself','his','how','i','if','in','into','is','isn',
             "isn't",'it',"it's",'its','itself','just','ll','m','ma','me','mightn',"mightn't",'more','most','mustn',"mustn't",'my','myself','needn',"needn't",'nor','now','o','of',
             'once','only','or','other','our','ours','ourselves','out','own','re','s','same','shan',"shan't",'she',"she's",'should',"should've",'shouldn',"shouldn't",'so','some',
             'such','t','than','that',"that'll",'the','their','theirs','them','themselves','then','there','these','they','this','those','through','to','too','until','ve','very',
             'was','wasn',"wasn't",'we','were','weren',"weren't",'what','when','where','which','while','who','whom','why','will','with','won',"won't",'wouldn',"wouldn't",'y','you',
             "you'd","you'll","you're","you've",'your','yours','yourself','yourselves']

def normalize_template(text):
  """
  Normalize text to extract most salient tokens
  """
  # replace special characters with space and remove digits
  text = re.sub(r'\W+', ' ', text) # replaces one or more non-alphanumeric characters (\W+) with a single space in the text
  text = re.sub('\d', '', text)    #  replaces any digit (\d) with an empty string in the text

  # convert camel case to snake case, then replace _ with space
  text = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', text)
  text = re.sub('([a-z0-9])([A-Z])', r'\1_\2', text).lower().replace('_', ' ')

  normalized_tokens = [w for w in text.split() if w not in stop_words]

  # Reconstruct the sentence
  filtered_sentence = ' '.join(normalized_tokens) # our output will be sentences not a list of words

  return filtered_sentence

In [41]:
normalized_templates = [normalize_template(sentence) for sentence in df['EventTemplate'] ]
#normalized_templates[0] = ' '
print(normalized_templates)

['', 'bglerr ido pkt timeout connection lost node link service card', 'correctable errors exceeds threshold iar lr', 'ddr error detected corrected on rank symbol over seconds', 'ddr errors detected corrected on rank symbol bit', 'double hummer alignment exceptions', 'exited abnormally due signal aborted', 'exited normally exit code', 'floating point alignment exceptions', 'l error dcr detected corrected over seconds', 'l error dcr detected corrected', 'microseconds spent rbs signal handler calls microseconds maximum time single instance correctable ddr', 'pgood error latched on link card', 'power module not accessible', 'tlb error interrupt', 'torus non crc error dcr detected corrected over seconds', 'torus non crc error dcr detected corrected', 'torus receiver input pipe error dcr detected corrected', 'torus receiver input pipe error dcr detected corrected over seconds', 'torus error dcr detected corrected', 'torus error dcr detected corrected over seconds', 'total interrupts critical

PART1 : Using light sentence-transformers model

In [7]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [42]:
dic_templates = {}

for i, sentence in enumerate(normalized_templates):
  dic_templates[i] = model.encode(sentence)

In [34]:
dic_templates[0]

array([-1.18838415e-01,  4.82985973e-02, -2.54803943e-03, -1.10112289e-02,
        5.19507490e-02,  1.02917394e-02,  1.15433216e-01,  7.00848235e-04,
       -8.59253332e-02, -7.06540421e-02,  1.33173075e-03, -3.54723595e-02,
        1.84341781e-02, -6.73720939e-03,  2.44029872e-02, -2.95032095e-02,
       -5.81384152e-02, -5.04395179e-02, -2.07654592e-02,  2.90360246e-02,
       -6.36760071e-02,  2.40298584e-02,  2.62433160e-02, -6.03737170e-03,
       -1.10765230e-02, -1.40070624e-03, -1.86198391e-02,  3.27701010e-02,
        2.88602803e-03, -5.69439717e-02, -4.39416654e-02,  2.54141018e-02,
        8.79094452e-02, -2.49912161e-02, -3.66832726e-02,  6.24129921e-03,
       -6.64679930e-02, -6.71445280e-02,  2.05642357e-02,  4.23888043e-02,
        2.18802709e-02, -4.28824760e-02, -3.43769975e-02,  6.14668466e-02,
        6.56373054e-02, -7.85202980e-02,  2.94869766e-02,  1.07983518e-02,
        6.33240938e-02, -4.50846516e-02, -1.82339456e-02, -2.77210828e-02,
       -3.67376721e-03, -

In [43]:
def cosine_similarity(vector1, vector2):
    # Calculate the dot product
    dot_product = np.dot(vector1, vector2)

    # Calculate the magnitudes
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)

    # Calculate the cosine similarity
    cosine_similarity = dot_product / (magnitude1 * magnitude2)

    return cosine_similarity



In [44]:
normalized_templates[5]

'double hummer alignment exceptions'

In [45]:
for i in range(30):
  print("Similarity:",cosine_similarity(dic_templates[6], dic_templates[i]))
  print(normalized_templates[6])
  print(normalized_templates[i])
  print('------------------------------------')

Similarity: 0.008875
exited abnormally due signal aborted

------------------------------------
Similarity: 0.33189136
exited abnormally due signal aborted
bglerr ido pkt timeout connection lost node link service card
------------------------------------
Similarity: 0.27483937
exited abnormally due signal aborted
correctable errors exceeds threshold iar lr
------------------------------------
Similarity: 0.22748198
exited abnormally due signal aborted
ddr error detected corrected on rank symbol over seconds
------------------------------------
Similarity: 0.22752665
exited abnormally due signal aborted
ddr errors detected corrected on rank symbol bit
------------------------------------
Similarity: 0.12808149
exited abnormally due signal aborted
double hummer alignment exceptions
------------------------------------
Similarity: 1.0
exited abnormally due signal aborted
exited abnormally due signal aborted
------------------------------------
Similarity: 0.60399705
exited abnormally due 

In [46]:
for i in range(len(normalized_templates)):
  for j in range(i+1,len(normalized_templates)) :

      similarity = cosine_similarity(dic_templates[i], dic_templates[j])

      if similarity > 0.99 :
        print(i,j)
        print('similarity = ',similarity)
        print(normalized_templates[i])
        print(normalized_templates[j])
        print('--------------------------------')

85 86
similarity =  1.0000001
core configuration register
core configuration register
--------------------------------
132 133
similarity =  1.0000001
end service action performed upon
end service action performed upon
--------------------------------
145 146
similarity =  1.0000001
exception syndrome register
exception syndrome register
--------------------------------
165 166
similarity =  1.0
general purpose registers
general purpose registers
--------------------------------
199 350
similarity =  1.0
link card not fully functional
link card not fully functional
--------------------------------
212 213
similarity =  1.0000001
machine check status register
machine check status register
--------------------------------
214 215
similarity =  1.0000001
machine state register
machine state register
--------------------------------
214 216
similarity =  1.0000001
machine state register
machine state register
--------------------------------
215 216
similarity =  1.0000001
machine state re

PART2 : using word2vec and tff-idf